## 데이터 수집

- https://kadx.co.kr/opmk/frn/pmumkproductDetail/PMU_79c6f1a4-56dd-492e-ad67-c5acba0304d2/5# 에서 무료 제공
- csv파일 3개 다운로드

## 데이터 전처리

- 인코딩 utf-8

In [ ]:
import pandas as pd
import chardet

# 기존 파일 리스트
original_paths = [
    # "TB_RECIPE_SEARCH-220701.csv",
    # "TB_RECIPE_SEARCH-20231130.csv",
    "TB_RECIPE_SEARCH_241226.csv"
]

def encoding(filepath, num_bytes=10000):
    # 인코딩 감지
    with open(filepath, 'rb') as f:
        raw = f.read(num_bytes)
    result = chardet.detect(raw)
    return result['encoding']

# 인코딩 변환 및 덮어쓰기 저장
for path in original_paths:
    try:
        detected_encoding = encoding(path)
        print(f"[감지됨] {path} → 인코딩: {detected_encoding}")

        # 감지된 인코딩으로 열기
        with open(path, 'r', encoding=detected_encoding, errors='replace') as f:
            df = pd.read_csv(f)

        # UTF-8 인코딩 덮어쓰기 저장
        df.to_csv(path, index=False, encoding='utf-8-sig')

        print(f"[저장 완료] {path} → UTF-8 인코딩 완료")

    except Exception as e:
        print(f"[실패] {path}: {e}")

[감지됨] TB_RECIPE_SEARCH-20231130.csv → 인코딩: EUC-KR
[저장 완료] TB_RECIPE_SEARCH-20231130.csv → UTF-8 인코딩 완료
